In [1]:
%matplotlib inline
%config InlineBackend.figure_format = "retina"

In [2]:
# Author: xiebo

# Python implementation: CPython
# Python version       : 3.8.5
# IPython version      : 8.12.3

# torch      : 2.2.0+cu118
# torchdata  : 0.7.1
# torchtext  : 0.17.0
# torchvision: 0.17.0+cu118
# torchaudio : 2.2.0+cu118
# openai     : 1.10.0
# langchain  : 0.1.5
# tiktoken   : 0.5.2
# numpy      : 1.24.4
# pandas     : 2.0.3
# sklearn    : 1.3.2
# matplotlib : 3.7.4
# scipy      : 1.10.1

%load_ext watermark
%watermark -a 'xiebo' -d -t -v -p torch,torchdata,torchtext,torchvision,torchaudio,openai,langchain,tiktoken,numpy,pandas,sklearn,matplotlib,scipy

Author: xiebo

Python implementation: CPython
Python version       : 3.8.5
IPython version      : 8.12.3

torch      : 2.2.0+cu118
torchdata  : 0.7.1
torchtext  : 0.17.0
torchvision: 0.17.0+cu118
torchaudio : 2.2.0+cu118
openai     : 1.10.0
langchain  : 0.1.5
tiktoken   : 0.5.2
numpy      : 1.24.4
pandas     : 2.0.3
sklearn    : 1.3.2
matplotlib : 3.7.4
scipy      : 1.10.1



In [3]:
import sys
sys.path.append("..")
sys.path.append("../..")

from project_utils import *

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
fix_all_seed()

In [6]:
print_closeai()

OPENAI_BASE_URL: https://api.closeai-proxy.xyz/v1
OPENAI_API_KEY: 8V55bZBNNrS0keILgUUIQwW1RWO-Ys4siyk9zwk2i27DwqoCzaQ


In [7]:
client = get_openai_client()

In [8]:
#### 定义任务描述和输入
# 测试LLM
# 任务描述
instruction = """
你的任务是识别用户对手机流量套餐产品的选择条件。
每种流量套餐产品包含三个属性：名称，月费价格，月流量。
根据用户输入，识别用户在上述三种属性上的倾向。
"""

# 用户输入
input_text = """
办个100G的套餐。
"""

# prompt 模版。instruction 和 input_text 会被替换为上面的内容
prompt = f"""
{instruction}

用户输入：
{input_text}
"""

# prompt_token_count = 94, completion_token_count = 77, total_token_count = 171.
# choice_count = 1, cost time = 2.4s, finish_reason = stop.
# 用户在流量套餐产品的选择条件上的倾向为：
# - 名称：用户倾向选择100G的套餐。
# - 月费价格：用户未提及对月费价格的倾向。
# - 月流量：用户倾向选择100G的套餐。
content = get_chat_completion_content(client, prompt)
print(content)

prompt_token_count = 94, completion_token_count = 34, total_token_count = 128.
choice_count = 1, cost time = 1.6s, finish_reason = stop.

用户倾向：
- 名称：100G套餐
- 月费价格：不确定
- 月流量：100G


In [9]:
# 从上面的结果可以看出 LLM 理解了我们的需求。
# 但是自然语言的输出，不方便我们形成最终的决策。

# 因此我们需要对输出的格式进行约定，形成格式化的输出，**推荐使用 JSON 格式输出**

# 输出格式限定
output_format = """
以 JSON 格式输出
"""

# 加入输出格式
prompt = f"""
{instruction}

{output_format}

用户输入：
{input_text}
"""

# {
#   "名称": "100G套餐",
#   "月费价格": "未知",
#   "月流量": "100G"
# }
content = get_chat_completion_content(client, prompt)
print(content)

prompt_token_count = 100, completion_token_count = 32, total_token_count = 132.
choice_count = 1, cost time = 1.3s, finish_reason = stop.

{
  "名称": "100G套餐",
  "月费价格": "",
  "月流量": "100G"
}


In [10]:
# 任务描述增加了字段的**英文标识符**
instruction = """
你的任务是识别用户对手机流量套餐产品的选择条件。
每种流量套餐产品包含三个属性：名称(name)，月费价格(price)，月流量(data)。
根据用户输入，识别用户在上述三种属性上的倾向。
"""

# 输出格式增加了各种定义、约束
output_format = """
以JSON格式输出。
1. name字段的取值为string类型，取值必须为以下之一：经济套餐、畅游套餐、无限套餐、校园套餐 或 null；

2. price字段的取值为一个结构体 或 null，包含两个字段：
(1) operator, string类型，取值范围：'<='（小于等于）, '>=' (大于等于), '=='（等于）
(2) value, int类型

3. data字段的取值为取值为一个结构体 或 null，包含两个字段：
(1) operator, string类型，取值范围：'<='（小于等于）, '>=' (大于等于), '=='（等于）
(2) value, int类型或string类型，string类型只能是'无上限'

4. 用户的意图可以包含按price或data排序，以sort字段标识，取值为一个结构体：
(1) 结构体中以"ordering"="descend"表示按降序排序，以"value"字段存储待排序的字段
(2) 结构体中以"ordering"="ascend"表示按升序排序，以"value"字段存储待排序的字段

只输出中只包含用户提及的字段，不要猜测任何用户未直接提及的字段，不输出值为null的字段。
"""

# {
#   "data": {
#     "operator": ">=",
#     "value": 100
#   }
# }
# input_text = "办个100G以上的套餐"

# {
#   "name": "无限套餐"
# }
# input_text = "我要无限量套餐"

# {
#   "name": "经济套餐",
#   "price": {
#     "operator": "<=",
#     "value": 50
#   }
# }
# input_text = "有没有便宜的套餐"

# {
#   "price": {
#     "operator": "<=",
#     "value": 100
#   },
#   "data": {
#     "operator": ">=",
#     "value": 10
#   },
#   "sort": {
#     "ordering": "descend",
#     "value": "data"
#   }
# }
# input_text = "有没有便宜的套餐，但流量能多一些"

# {
#   "data": {
#     "operator": ">=",
#     "value": "无上限"
#   }
# }
input_text = "我只要流量足够多的套餐"

prompt = f"""
{instruction}

{output_format}

用户输入：
{input_text}
"""

content = get_chat_completion_content(client, prompt)
print(content)

prompt_token_count = 454, completion_token_count = 25, total_token_count = 479.
choice_count = 1, cost time = 0.9s, finish_reason = stop.

{
  "data": {
    "operator": ">=",
    "value": "无上限"
  }
}


In [11]:
examples = """
便宜的套餐：{"sort":{"ordering"="ascend","value"="price"}}
有没有不限流量的：{"data":{"operator":"==","value":"无上限"}}
流量大的：{"sort":{"ordering"="descend","value"="data"}}
100G以上流量的套餐最便宜的是哪个：{"sort":{"ordering"="ascend","value"="price"},"data":{"operator":">=","value":100}}
月费不超过200的：{"price":{"operator":"<=","value":200}}
就要月费180那个套餐：{"price":{"operator":"==","value":180}}
经济套餐：{"name":"经济套餐"}
"""

# {"sort":{"ordering":"ascend","value":"price"}}
# input_text = "有没有便宜的套餐"

# {"name":"无限套餐"}
# input_text = "有没有土豪套餐"

# {"data":{"operator":">=","value":200},"sort":{"ordering":"ascend","value":"price"}}
# input_text = "办个200G的套餐"

# {"sort":{"ordering":"descend","value":"data"}}
# input_text = "有没有流量大的套餐"

# {"price":{"operator":"<=","value":200},"sort":{"ordering":"descend","value":"data"}}
# input_text = "200元以下，流量大的套餐有啥"

# {"data":{"operator":"==","value":10}}
input_text = "你说那个10G的套餐，叫啥名字"

# 有了例子
prompt = f"""
{instruction}

{output_format}

例如：
{examples}

用户输入：
{input_text}

"""

content = get_chat_completion_content(client, prompt)
print(content)

prompt_token_count = 639, completion_token_count = 11, total_token_count = 650.
choice_count = 1, cost time = 0.7s, finish_reason = stop.

{"data":{"operator":"==","value":10}}


In [12]:
instruction = """
你的任务是识别用户对手机流量套餐产品的选择条件。
每种流量套餐产品包含三个属性：名称(name)，月费价格(price)，月流量(data)。
根据对话上下文，识别用户在上述属性上的倾向。识别结果要包含整个对话的信息。
"""

# 输出描述
output_format = """
以JSON格式输出。
1. name字段的取值为string类型，取值必须为以下之一：经济套餐、畅游套餐、无限套餐、校园套餐 或 null；

2. price字段的取值为一个结构体 或 null，包含两个字段：
(1) operator, string类型，取值范围：'<='（小于等于）, '>=' (大于等于), '=='（等于）
(2) value, int类型

3. data字段的取值为取值为一个结构体 或 null，包含两个字段：
(1) operator, string类型，取值范围：'<='（小于等于）, '>=' (大于等于), '=='（等于）
(2) value, int类型或string类型，string类型只能是'无上限'

4. 用户的意图可以包含按price或data排序，以sort字段标识，取值为一个结构体：
(1) 结构体中以"ordering"="descend"表示按降序排序，以"value"字段存储待排序的字段
(2) 结构体中以"ordering"="ascend"表示按升序排序，以"value"字段存储待排序的字段

只输出中只包含用户提及的字段，不要猜测任何用户未直接提及的字段。不要输出值为null的字段。
"""
# DO NOT OUTPUT NULL-VALUED FIELD!

# 多轮对话的例子
examples = """
客服：有什么可以帮您
用户：100G套餐有什么

{"data":{"operator":">=","value":100}}

客服：有什么可以帮您
用户：100G套餐有什么
客服：我们现在有无限套餐，不限流量，月费300元
用户：太贵了，有200元以内的不

{"data":{"operator":">=","value":100},"price":{"operator":"<=","value":200}}

客服：有什么可以帮您
用户：便宜的套餐有什么
客服：我们现在有经济套餐，每月50元，10G流量
用户：100G以上的有什么

{"data":{"operator":">=","value":100},"sort":{"ordering"="ascend","value"="price"}}

客服：有什么可以帮您
用户：100G以上的套餐有什么
客服：我们现在有畅游套餐，流量100G，月费180元
用户：流量最多的呢

{"sort":{"ordering"="descend","value"="data"},"data":{"operator":">=","value":100}}
"""

# {"sort":{"ordering"="ascend","value"="price"},"data":{"operator":">=","value":100}}
# input_text = "哪个便宜"

# {"name": "无限套餐"}
# input_text = "无限量哪个多少钱"

# {"sort":{"ordering"="descend","value"="data"}}
input_text = "流量最大的多少钱"

# 多轮对话上下文
context = f"""
客服：有什么可以帮您
用户：有什么100G以上的套餐推荐
客服：我们有畅游套餐和无限套餐，您有什么价格倾向吗
用户：{input_text}
"""

prompt = f"""
{instruction}

{output_format}

{examples}

{context}
"""

content = get_chat_completion_content(client, prompt)
print(content)

prompt_token_count = 836, completion_token_count = 14, total_token_count = 850.
choice_count = 1, cost time = 1.1s, finish_reason = stop.

{"sort":{"ordering"="descend","value"="data"}}


In [13]:
# 1 system:
# 	你是一个手机流量套餐的客服代表。可以帮助用户选择最合适的流量套餐产品。可以选择的套餐包括：
# 经济套餐，月费50元，10G流量；
# 畅游套餐，月费180元，100G流量；
# 无限套餐，月费300元，1000G流量；
# 校园套餐，月费150元，200G流量，仅限在校生。
# 1 user:
# 	有没有土豪套餐？
# 1 assistant:
# 	很抱歉，我们没有土豪套餐。但是我们有无限套餐，它提供1000G的流量，适合大流量用户。
# 2 user:
# 	多少钱？
# 2 assistant:
# 	无限套餐的月费是300元。
# 3 user:
# 	给我办一个？
# 3 assistant:
# 	好的，请问您是在校生吗？

history_message_list = list()

system_prompt = """
你是一个手机流量套餐的客服代表。可以帮助用户选择最合适的流量套餐产品。可以选择的套餐包括：
经济套餐，月费50元，10G流量；
畅游套餐，月费180元，100G流量；
无限套餐，月费300元，1000G流量；
校园套餐，月费150元，200G流量，仅限在校生。
"""

get_chat_completion_content(client, user_prompt="有没有土豪套餐？", system_prompt=system_prompt, history_message_list=history_message_list)
get_chat_completion_content(client, user_prompt="多少钱？",  history_message_list=history_message_list)
get_chat_completion_content(client, user_prompt="给我办一个？", history_message_list=history_message_list)

print_history_message_list(history_message_list)

prompt_token_count = 158, completion_token_count = 49, total_token_count = 207.
choice_count = 1, cost time = 1.5s, finish_reason = stop.

prompt_token_count = 220, completion_token_count = 14, total_token_count = 234.
choice_count = 1, cost time = 0.7s, finish_reason = stop.

prompt_token_count = 248, completion_token_count = 13, total_token_count = 261.
choice_count = 1, cost time = 0.8s, finish_reason = stop.

1 system:
	你是一个手机流量套餐的客服代表。可以帮助用户选择最合适的流量套餐产品。可以选择的套餐包括：
经济套餐，月费50元，10G流量；
畅游套餐，月费180元，100G流量；
无限套餐，月费300元，1000G流量；
校园套餐，月费150元，200G流量，仅限在校生。
1 user:
	有没有土豪套餐？
1 assistant:
	很抱歉，我们没有土豪套餐。但是我们有无限套餐，它提供1000G的流量，适合大流量用户。
2 user:
	多少钱？
2 assistant:
	无限套餐的月费是300元。
3 user:
	给我办一个？
3 assistant:
	好的，请问您是在校生吗？


In [14]:
def abc():

SyntaxError: unexpected EOF while parsing (<ipython-input-14-a63b37fd44e1>, line 1)

In [ ]:
# prompt_token_count = 9, completion_token_count = 18, total_token_count = 27.
# choice_count = 1, cost time = 1.2s, finish_reason = stop.
# 你好！有什么我可以帮助你的吗？
content = get_chat_completion_content(client, "你好")
print(content)

In [ ]:
# prompt_token_count = 67, completion_token_count = 379, total_token_count = 446.
# choice_count = 1, cost time = 6.3s, finish_reason = stop.
# 作为私人秘书，我可以帮您处理日常事务，包括但不限于以下任务：

# 1. 管理日程安排：安排会议、预订行程、提醒重要事件等。
# 2. 筛选和回复邮件：帮您处理电子邮件，筛选重要邮件并回复。
# 3. 组织文件和资料：整理和管理文件、文档和资料，确保您能够轻松找到所需信息。
# 4. 安排会议和活动：协助您安排会议、活动和社交聚会，包括预订场地、发送邀请函等。
# 5. 电话和邮件代接：代接您的电话和邮件，过滤垃圾信息并转达重要信息。
# 6. 旅行安排：为您预订机票、酒店和交通工具，确保您的出行顺利。
# 7. 财务管理：协助您管理个人财务，包括账单支付、预算制定等。
# 8. 管理社交媒体：帮您管理社交媒体账号，发布内容、回复评论等。

# 我全天候无休假，根据您的需求，我可以根据您的时间安排灵活上班。无论是早晨、下午还是晚上，我都会为您提供服务。
content = get_chat_completion_content(client,
                            user_prompt="你可以干啥？你什么时间上班",
                            system_prompt="你是一个私人秘书，你可以帮我干很多杂事，你为我工作的时间是全天候无休假。",
                            temperature=0.2)
print(content)

In [ ]:
# prompt_token_count = 29, completion_token_count = 57, total_token_count = 86.
# choice_count = 1, cost time = 1.4s, finish_reason = stop.
# 作为一个人工智能助手，我没有性别和年龄。我只是一个程序，专注于提供帮助和回答问题。有什么我可以帮助你的吗？
content = get_chat_completion_content(client,
                            user_prompt=["你是男生还是女生?", "你的年纪是多大？"],
                            temperature=0.8)
print(content)